In [33]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import h5py
import torch
import numpy as np
from pytorch3dunet.unet3d.losses import *
from pathlib import Path
import glob
import seaborn as sns

ModuleNotFoundError: No module named 'seaborn'

In [24]:
loss = BCEDiceLoss(1.0,1.0)
loss1 = DiceLoss()
loss2 = torch.nn.BCEWithLogitsLoss()

loss

BCEDiceLoss(
  (bce): BCEWithLogitsLoss()
  (dice): DiceLoss(
    (normalization): Sigmoid()
  )
)

In [11]:
basepred = Path('/home/lorenzo/3dunet-cavity/runs/run_210601_local/predictions')
baseorig = Path('/home/lorenzo/deep_apbs/destData/pdbbind_v2013_core_set_0') # /2yfe/2yfe_grids.h5'

def genDataSets():

    for predfname in glob.glob(str(basepred / '*_grids_predictions.h5')):
        name = Path(predfname).name.split('_')[0]
        labelfname = baseorig / name / f'{name}_grids.h5'
        
        labelT = torch.tensor(h5py.File(labelfname)['label'], dtype=torch.float32)
        labelT = labelT[None,None]
        predT = torch.tensor(h5py.File(predfname)['predictions'])
        predT = predT[None]
        
        yield (predT, labelT)

In [28]:
class RunningAverage:
    def __init__(self):
        self.count = 0
        self.sum = 0

    def update(self, value):
        self.count += 1
        self.sum += value
        
    def value(self):
        return self.sum / self.count

OracleLoss = RunningAverage()
UnetLoss = RunningAverage()
RandLoss = RunningAverage()
RandUnitLoss = RunningAverage()
ZeroLoss = RunningAverage()
UnitLoss = RunningAverage()

for predT, labelT in genDataSets():
    constPred = torch.zeros_like(labelT)
    ZeroLoss.update(loss(constPred, labelT).item())
    
    constPred[:] = 1
    UnitLoss.update(loss(constPred, labelT).item())
    
    l = loss(predT, labelT).item()
    UnetLoss.update(l)
    
    l = loss(labelT, labelT).item()
    l1 = loss1(labelT, labelT).item()
    l2 = loss2(labelT, labelT).item()
    print(l,l1,l2)
    OracleLoss.update(l)

    randPred = torch.rand(size=constPred.size(), dtype=constPred.dtype, device=constPred.device, requires_grad=False)
    RandLoss.update(loss(randPred, labelT).item())
    
    randPred[randPred < 0.5] = 0
    randPred[randPred > 0.5] = 1
    RandUnitLoss.update(loss(randPred, labelT).item())

1.6853091716766357 0.9926428198814392 0.6926664113998413
1.6845664978027344 0.9919460415840149 0.6926205158233643
1.6879982948303223 0.9951661229133606 0.6928321123123169
1.685294508934021 0.9926289916038513 0.6926655173301697
1.6820738315582275 0.9896076321601868 0.6924661993980408
1.6874299049377441 0.9946327805519104 0.6927971243858337
1.6884751319885254 0.9956137537956238 0.6928614377975464
1.6850290298461914 0.9923800230026245 0.6926490664482117
1.6842854022979736 0.991682231426239 0.6926031112670898
1.6875689029693604 0.9947633147239685 0.6928056478500366
1.6860380172729492 0.9933266043663025 0.6927114129066467
1.685973048210144 0.993265688419342 0.692707359790802
1.6870070695877075 0.9942359328269958 0.6927711367607117


In [29]:
predT, labelT = next(genDataSets())

In [32]:
labelT.mean(), labelT.std()

(tensor(0.0013), tensor(0.0356))

In [20]:
RandUnitLoss.value()

1.9989742132333608

In [14]:
RandLoss.value()

1.979505914908189

In [15]:
UnitLoss.value()

2.3089121671823354

In [16]:
ZeroLoss.value()

1.688505429487962

In [17]:
UnetLoss.value()

1.688859646136944

In [18]:
OracleLoss.value()

1.6859268316855798

## Scratch

In [180]:
name = '1z95'

dstbase = '/home/lorenzo/3dunet-cavity/npy'

predfname = f'/home/lorenzo/3dunet-cavity/runs/run_210601/predictions/{name}_grids_predictions.h5'
origfname = f'/home/lorenzo/deep_apbs/destData/pdbbind_v2013_core_set_0/{name}/{name}_grids.h5'


vlabel = np.array(h5py.File(origfname)['label'])
with open(f'{dstbase}/{name}_label.npy','wb') as f:
    np.save(f, vlabel)

# h5py.File(predbase)['predictions']


array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 